In [1]:
import pandas as pd
import re
import numpy as np 
import matplotlib.pyplot as plt 
#!pip install seaborn
import seaborn as sns
import string
#!pip install nltk
import nltk
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

%matplotlib inline

In [2]:
df = pd.read_csv('train_tweets.csv',encoding = 'latin',names=["target","id","date","Query","user","tweet"])

In [3]:
#Partir el dataset
df['preprocess']=''
df.head()
df1 = df.loc[:200000,:]
df2 = df.loc[400000:799999,:]
df3 = df.loc[800000:1199999,:]
df4 = df.loc[1200000:,:]
print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)

(200001, 7)
(12234, 7)
(0, 7)
(0, 7)


In [4]:
grouped = df.groupby("target")

In [5]:
grouped.count()["id"]

target
0    412234
Name: id, dtype: int64

In [33]:
#Limpieza basica del tweet
def processTweet(tweet):
    # process the tweets
    #Convert to lower case
    tweet = tweet.str.lower()
    #Convert www.* or https?://* to URL
    tweet = tweet.str.replace('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', regex=True)
    #tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Convert @username to AT_USER
    tweet = tweet.str.replace('@[^\s]+', 'AT_USER', regex=True)
    #tweet = re.sub('@[^\s]+','AT_USER',tweet)
    #Remove additional white spaces
    tweet = tweet.str.replace('[\s]+', ' ', regex=True)
    #tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = tweet.str.replace(r'#([^\s]+)', r'\1', regex=True)
    #tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #tweet = tweet.str.replace("[^a-zA-Z#]","SPECIAL")
    #trim
    tweet = tweet.str.strip('\'"')
    return tweet

df1.loc[:,'preprocess'] =processTweet(df1.loc[:,'tweet'])
df2.loc[:,'preprocess'] =processTweet(df2.loc[:,'tweet'])
df3.loc[:,'preprocess'] =processTweet(df3.loc[:,'tweet'])

In [7]:
#JK: YA no se requiere
#Metodo que remueve un patrón usado para quitar las menciones @user
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, 'AT_USER', input_txt)    
    return input_txt 

In [8]:
#Se elminan las menciones del dataset
#df1.loc[:,'preprocess'] = np.vectorize(remove_pattern)(df1['tweet'], "@[\w]*")

In [9]:
#df2.loc[:,'preprocess'] = np.vectorize(remove_pattern)(df2['tweet'], "@[\w]*")
#df3.loc[:,'preprocess'] = np.vectorize(remove_pattern)(df3['tweet'], "@[\w]*")
#df4.loc[:,'preprocess'] = np.vectorize(remove_pattern)(df4['tweet'], "@[\w]*")

In [32]:
#SE USA PARA VISUALIZAR EL DATASET ANTES Y DESPUES DEL PREPROCESS
df1.head().loc[:,"tweet":"preprocess"]

,tweet,preprocess
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","ATUSER URL - awww, that's a bummer. you should..."
1,is upset that he can't update his Facebook by ...,is upset that he can't update his facebook by ...
2,@Kenichan I dived many times for the ball. Man...,ATUSER i dived many times for the ball. manage...
3,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire
4,"@nationwideclass no, it's not behaving at all....","ATUSER no, it's not behaving at all. i'm mad. ..."


In [11]:
df2.head()

,target,id,date,Query,user,tweet,preprocess
400000,0,2057298217,Sat Jun 06 12:44:31 PDT 2009,NO_QUERY,kannibalKoi,so mutha effin bored,so mutha effin bored
400001,0,2057298537,Sat Jun 06 12:44:33 PDT 2009,NO_QUERY,TMankin,@CallaLilies83 not at all...I've spent most of...,AT_USER not at all...i've spent most of the af...
400002,0,2057298663,Sat Jun 06 12:44:34 PDT 2009,NO_QUERY,theresa_jx,@GiGisOssum :/ My entire friggin family? Ther...,AT_USER :/ my entire friggin family? there's n...
400003,0,2057298745,Sat Jun 06 12:44:34 PDT 2009,NO_QUERY,mishellfish,I absolutely &lt;3 sleeping in on saturdays. I...,i absolutely &lt;3 sleeping in on saturdays. i...
400004,0,2057298766,Sat Jun 06 12:44:34 PDT 2009,NO_QUERY,meganleahhall,is a little upset at the moment,is a little upset at the moment


In [12]:
df3.head()

,target,id,date,Query,user,tweet,preprocess


In [13]:
df4.head()

,target,id,date,Query,user,tweet,preprocess


In [14]:
#Se remueven las url
#df1.loc[:,'preprocess'] = df1.loc[:,'preprocess'].str.replace("https\S+|http\S+|www.\S+","",case = False)
#Se remueven las apostrofes
#df1.loc[:,'preprocess'] = df1.loc[:,'preprocess'].str.replace("'","")
#Se remueven los caracteres especiales
#df1.loc[:,'preprocess'] = df1.loc[:,'preprocess'].str.replace("[^a-zA-Z#]"," ")

In [15]:
#Se elminan todas las palabras de menos de 3 letras
#df1.loc[:,'preprocess'] = df1.loc[:,'preprocess'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [16]:
#Se convierten las palabras el tweet a elementos de un arreglo
tokenized_tweet = df1.loc[:,'preprocess'].apply(lambda x: x.split())
tokenized_tweet.head()

0    [AT_USER, URL, -, awww,, that's, a, bummer., y...
1    [is, upset, that, he, can't, update, his, face...
2    [AT_USER, i, dived, many, times, for, the, bal...
3    [my, whole, body, feels, itchy, and, like, its...
4    [AT_USER, no,, it's, not, behaving, at, all., ...
Name: preprocess, dtype: object

In [17]:
from nltk.stem.porter import *
#stemmer = PorterStemmer()
#Librerìa para manejar los verbos en sus distintas formas (falta validar si es buena opción)
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) # stemming
tokenized_tweet.head()

0    [at_us, url, -, awww,, that, a, bummer., you, ...
1    [is, upset, that, he, can't, updat, his, faceb...
2    [at_us, i, dive, mani, time, for, the, ball., ...
3    [my, whole, bodi, feel, itchi, and, like, it, ...
4    [at_us, no,, it, not, behav, at, all., i'm, ma...
Name: preprocess, dtype: object

In [18]:
#Despues de transformar los verbos, los ponemos en el dataset
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])

df1.loc[:,'preprocess'] = tokenized_tweet

In [19]:
df1.loc[:,"tweet":'preprocess']

,tweet,preprocess
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","at_us url - awww, that a bummer. you shoulda g..."
1,is upset that he can't update his Facebook by ...,is upset that he can't updat his facebook by t...
2,@Kenichan I dived many times for the ball. Man...,at_us i dive mani time for the ball. manag to ...
3,my whole body feels itchy and like its on fire,my whole bodi feel itchi and like it on fire
4,"@nationwideclass no, it's not behaving at all....","at_us no, it not behav at all. i'm mad. whi am..."
5,@Kwesidei not the whole crew,at_us not the whole crew
6,Need a hug,need a hug
7,@LOLTrish hey long time no see! Yes.. Rains a...,"at_us hey long time no see! yes.. rain a bit ,..."
8,@Tatiana_K nope they didn't have it,at_us nope they didn't have it
9,@twittera que me muera ?,at_us que me muera ?


In [20]:
#!pip install wordcloud
#Graficar las palabras en una nube de palabras, actualmente no me deja instalar la librería por memoria
all_words = ' '.join([text for text in df1.loc[:,'preprocess']])
from wordcloud import WordCloud
wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(all_words)

plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

ModuleNotFoundError: No module named 'wordcloud'

In [ ]:
!pip install wordcloud

In [ ]:
!sudo swapon -s